In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
os.getcwd()

'/Users/miek/Documents/DS4B/DS4BThesisSNCB'

In [3]:
stops['COMMERCIAL_STOP_NATURE'].value_counts()

COMMERCIAL_STOP_NATURE
2    48662742
1     4943900
3     4692222
Name: count, dtype: int64

In [2]:
# Read tjj
stops = pd.read_parquet("~/Downloads/TJJ_PTCAR_PARQUET (ALL COMM. SNCB TRAINS)/testtjjgent2")
stops = stops[stops["REF_REPERAGE_NO"] > 0]
stops

,REF_TOD_TRAIN_ID,TRAIN_PREFIX,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,REF_REPERAGE_NO,DEPARTURE_AT_DT_ORIGIN,ARRIVAL_AT_DT_ORIGIN,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR
1,22956975,None,2967,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 17:24:00,1900-01-01 00:00:00,2019-02-28 17:24:28,0,28,1,2019-02-28 17:24:00,1900-01-01 00:00:00,1,1410,59.0,59/1
3,23030624,None,2968,2019-03-13,37,101,1900-01-01 00:00:00,2019-03-13 18:24:00,1900-01-01 00:00:00,2019-03-13 18:24:25,0,25,1,2019-03-13 18:24:00,1900-01-01 00:00:00,1,1410,59.0,59/1
6,22787493,None,3343,2019-01-30,37,101,1900-01-01 00:00:00,2019-01-30 22:38:00,1900-01-01 00:00:00,2019-01-30 22:38:35,0,35,1,2019-01-30 22:38:00,1900-01-01 00:00:00,1,889,25.0,25
9,22957754,None,2868,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 18:57:00,1900-01-01 00:00:00,2019-02-28 18:57:24,0,24,1,2019-02-28 18:57:00,1900-01-01 00:00:00,1,1410,59.0,59/1
10,22804611,None,5944,2019-02-01,641,5,2019-02-01 00:20:00,2019-02-01 00:22:00,2019-02-01 00:20:41,2019-02-01 00:22:41,41,41,1,2019-02-01 00:22:00,2019-02-01 00:20:00,2,0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61345681,31329661,None,2627,2022-10-28,715,117,2022-10-28 06:13:00,2022-10-28 06:24:00,2022-10-28 06:13:49,2022-10-28 06:24:16,49,16,1,2022-10-28 06:24:00,2022-10-28 06:13:00,2,276,53.0,53
61345682,31325182,None,2131,2022-10-28,70,103,2022-10-28 08:31:00,2022-10-28 08:36:00,2022-10-28 08:28:29,2022-10-28 08:36:30,-150,30,1,2022-10-28 08:36:00,2022-10-28 08:31:00,2,532,162.0,162
61345683,31326770,None,5468,2022-10-28,726,106,2022-10-28 19:02:00,2022-10-28 19:06:00,2022-10-28 19:28:05,2022-10-28 19:29:49,1565,1429,1,2022-10-28 19:06:00,2022-10-28 19:02:00,2,259,37.0,37A
61345684,31330053,None,555,2022-10-28,455,115,2022-10-28 06:12:00,2022-10-28 06:19:00,2022-10-28 06:14:13,2022-10-28 06:19:58,133,58,1,2022-10-28 06:19:00,2022-10-28 06:12:00,2,647,50.0,50E


In [3]:
# add year column
stops["YEAR_DEP"] = stops["TRAIN_DEPARTURE_DATE"].dt.year

# intermediate df no. 1: stops filtered only on the final station per train
ending_stop = (stops.sort_values('THR_ARR', ascending=False)
               .drop_duplicates('REF_TOD_TRAIN_ID')
               .reset_index(drop=True)
)

# add column for datetime of the end of each train
ending_stop['end_lifetime'] = ending_stop['THR_ARR']

# create a new column with only the hour component of theoretical departure time
ending_stop['arr_hour_terminus'] = ending_stop['THR_ARR'].dt.hour + ending_stop['THR_ARR'].dt.minute / 60


# intermediate df no. 2: stops filtered only on departure station per train
starting_stop = (stops[(stops["THR_DEP"].dt.year != 1900)]
                 .sort_values('THR_DEP', ascending=True)
                 .drop_duplicates('REF_TOD_TRAIN_ID')
                 .reset_index(drop=True)
)

# add column for datetime of the start of each train
starting_stop['start_lifetime'] = starting_stop['THR_DEP']

# create a new column with only the hour component of theoretical departure time
starting_stop['dep_hour_origin'] = starting_stop['THR_DEP'].dt.hour + starting_stop['THR_DEP'].dt.minute / 60

# Create date column
starting_stop['date_origin'] = starting_stop['THR_DEP'].dt.date
starting_stop['date_origin'] = pd.to_datetime(starting_stop['date_origin'])

# Create month column
starting_stop['month'] = starting_stop['THR_DEP'].dt.month

# Create day column
starting_stop['day_origin'] = starting_stop['THR_DEP'].dt.day

# Create season column
starting_stop['season'] = starting_stop['month'].apply(lambda x:
                                                       'Winter' if x <= 2 else 'Spring' if x <= 5 else 'Summer' if x <= 8 else 'Fall' if x <= 11 else 'Winter')

# Create weekday/weekend column
starting_stop['weekend'] = np.where((starting_stop['THR_DEP']).dt.weekday < 5, 0, 1)

# Create Belgian national holiday dummy column
be_holidays = holidays.BE()
starting_stop['holiday'] = starting_stop['THR_DEP'].apply(lambda x: 1 if x in be_holidays else 0)




# merge origin departure time to all stops a train will make
stops = stops.merge(starting_stop[['REF_TOD_TRAIN_ID', 'month', 'season',
                                   'weekend', 'holiday', 'dep_hour_origin',
                                   'day_origin', 'date_origin', 'start_lifetime']],
                    on='REF_TOD_TRAIN_ID',
                    how='left')\
             .merge(ending_stop[['REF_TOD_TRAIN_ID', 'end_lifetime', 'arr_hour_terminus']],
                    on='REF_TOD_TRAIN_ID',
                    how='left')

# reduce columns of stops to useful features
stops = stops.drop(columns=["TRAIN_PREFIX", "REF_REPERAGE_NO", "DEPARTURE_AT_DT_ORIGIN", "ARRIVAL_AT_DT_ORIGIN"])

# add column for P-train
stops["p_indicator"] = stops['TRAIN_NUMBER'].apply(lambda x: 1 if ((x >= 7000) & (x <= 8999)) else 0)


# check
stops

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,...,season,weekend,holiday,dep_hour_origin,day_origin,date_origin,start_lifetime,end_lifetime,arr_hour_terminus,p_indicator
0,22956975,2967,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 17:24:00,1900-01-01 00:00:00,2019-02-28 17:24:28,0,...,Winter,0.0,0.0,17.400000,28.0,2019-02-28,2019-02-28 17:24:00,2019-02-28 18:25:00,18.416667,0
1,23030624,2968,2019-03-13,37,101,1900-01-01 00:00:00,2019-03-13 18:24:00,1900-01-01 00:00:00,2019-03-13 18:24:25,0,...,Spring,0.0,0.0,18.400000,13.0,2019-03-13,2019-03-13 18:24:00,2019-03-13 19:25:00,19.416667,0
2,22787493,3343,2019-01-30,37,101,1900-01-01 00:00:00,2019-01-30 22:38:00,1900-01-01 00:00:00,2019-01-30 22:38:35,0,...,Winter,0.0,0.0,22.633333,30.0,2019-01-30,2019-01-30 22:38:00,2019-01-30 23:33:00,23.550000,0
3,22957754,2868,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 18:57:00,1900-01-01 00:00:00,2019-02-28 18:57:24,0,...,Winter,0.0,0.0,18.950000,28.0,2019-02-28,2019-02-28 18:57:00,2019-02-28 20:02:00,20.033333,0
4,22804611,5944,2019-02-01,641,5,2019-02-01 00:20:00,2019-02-01 00:22:00,2019-02-01 00:20:41,2019-02-01 00:22:41,41,...,Winter,0.0,0.0,0.016667,1.0,2019-02-01,2019-02-01 00:01:00,2019-02-01 00:30:00,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58298859,31329661,2627,2022-10-28,715,117,2022-10-28 06:13:00,2022-10-28 06:24:00,2022-10-28 06:13:49,2022-10-28 06:24:16,49,...,Fall,0.0,0.0,5.616667,28.0,2022-10-28,2022-10-28 05:37:00,2022-10-28 07:12:00,7.200000,0
58298860,31325182,2131,2022-10-28,70,103,2022-10-28 08:31:00,2022-10-28 08:36:00,2022-10-28 08:28:29,2022-10-28 08:36:30,-150,...,Fall,0.0,0.0,8.600000,28.0,2022-10-28,2022-10-28 08:36:00,2022-10-28 11:28:00,11.466667,0
58298861,31326770,5468,2022-10-28,726,106,2022-10-28 19:02:00,2022-10-28 19:06:00,2022-10-28 19:28:05,2022-10-28 19:29:49,1565,...,Fall,0.0,0.0,18.650000,28.0,2022-10-28,2022-10-28 18:39:00,2022-10-28 19:23:00,19.383333,0
58298862,31330053,555,2022-10-28,455,115,2022-10-28 06:12:00,2022-10-28 06:19:00,2022-10-28 06:14:13,2022-10-28 06:19:58,133,...,Fall,0.0,0.0,5.516667,28.0,2022-10-28,2022-10-28 05:31:00,2022-10-28 07:28:00,7.466667,0


In [4]:
stops.dtypes

REF_TOD_TRAIN_ID                          int64
TRAIN_NUMBER                              int64
TRAIN_DEPARTURE_DATE             datetime64[ns]
REF_PTCAR_NO                              int64
PTCAR_ORD_NO                              int32
THR_ARR                          datetime64[ns]
THR_DEP                          datetime64[ns]
HR_ARR                           datetime64[ns]
HR_DEP                           datetime64[ns]
EH_ARR                                    int64
EH_DEP                                    int64
COMMERCIAL_STOP_NATURE                    int16
DEPARTURE_REF_LINE_SECTION_NO             int64
LINE                                    float64
IDENTIFICATOR                            object
YEAR_DEP                                  int32
month                                   float64
season                                   object
weekend                                 float64
holiday                                 float64
dep_hour_origin                         

In [5]:

stops.to_parquet("~/Downloads/all_stops_ungrouped/comm_stops_ungrouped.parquet")

In [6]:
stops

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,...,season,weekend,holiday,dep_hour_origin,day_origin,date_origin,start_lifetime,end_lifetime,arr_hour_terminus,p_indicator
0,22956975,2967,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 17:24:00,1900-01-01 00:00:00,2019-02-28 17:24:28,0,...,Winter,0.0,0.0,17.400000,28.0,2019-02-28,2019-02-28 17:24:00,2019-02-28 18:25:00,18.416667,0
1,23030624,2968,2019-03-13,37,101,1900-01-01 00:00:00,2019-03-13 18:24:00,1900-01-01 00:00:00,2019-03-13 18:24:25,0,...,Spring,0.0,0.0,18.400000,13.0,2019-03-13,2019-03-13 18:24:00,2019-03-13 19:25:00,19.416667,0
2,22787493,3343,2019-01-30,37,101,1900-01-01 00:00:00,2019-01-30 22:38:00,1900-01-01 00:00:00,2019-01-30 22:38:35,0,...,Winter,0.0,0.0,22.633333,30.0,2019-01-30,2019-01-30 22:38:00,2019-01-30 23:33:00,23.550000,0
3,22957754,2868,2019-02-28,37,101,1900-01-01 00:00:00,2019-02-28 18:57:00,1900-01-01 00:00:00,2019-02-28 18:57:24,0,...,Winter,0.0,0.0,18.950000,28.0,2019-02-28,2019-02-28 18:57:00,2019-02-28 20:02:00,20.033333,0
4,22804611,5944,2019-02-01,641,5,2019-02-01 00:20:00,2019-02-01 00:22:00,2019-02-01 00:20:41,2019-02-01 00:22:41,41,...,Winter,0.0,0.0,0.016667,1.0,2019-02-01,2019-02-01 00:01:00,2019-02-01 00:30:00,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58298859,31329661,2627,2022-10-28,715,117,2022-10-28 06:13:00,2022-10-28 06:24:00,2022-10-28 06:13:49,2022-10-28 06:24:16,49,...,Fall,0.0,0.0,5.616667,28.0,2022-10-28,2022-10-28 05:37:00,2022-10-28 07:12:00,7.200000,0
58298860,31325182,2131,2022-10-28,70,103,2022-10-28 08:31:00,2022-10-28 08:36:00,2022-10-28 08:28:29,2022-10-28 08:36:30,-150,...,Fall,0.0,0.0,8.600000,28.0,2022-10-28,2022-10-28 08:36:00,2022-10-28 11:28:00,11.466667,0
58298861,31326770,5468,2022-10-28,726,106,2022-10-28 19:02:00,2022-10-28 19:06:00,2022-10-28 19:28:05,2022-10-28 19:29:49,1565,...,Fall,0.0,0.0,18.650000,28.0,2022-10-28,2022-10-28 18:39:00,2022-10-28 19:23:00,19.383333,0
58298862,31330053,555,2022-10-28,455,115,2022-10-28 06:12:00,2022-10-28 06:19:00,2022-10-28 06:14:13,2022-10-28 06:19:58,133,...,Fall,0.0,0.0,5.516667,28.0,2022-10-28,2022-10-28 05:31:00,2022-10-28 07:28:00,7.466667,0
